In [1]:
%load_ext dotenv
%dotenv

!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


--2025-06-04 15:50:37--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8000::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0.001s  

2025-06-04 15:50:37 (6.73 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [6]:
import minsearch
import json
from openai import OpenAI

In [ ]:
with open('documents.json', 'rt') as file:
    docs_raw = json.load(file)

documents = []

for item in docs_raw:
    for doc in item["documents"]:
        doc['course'] = item['course']
        documents.append(doc)

documents

In [ ]:
search_engine = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

search_engine.fit(documents)


In [5]:
boost = {"question": 3.0, "section": 0.5}
question = "The course has already started, can I still enroll?"

search_results = search_engine.search(
    query=question,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
)

search_results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [9]:
prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
""".strip()

context = ""

for result in search_results:
    context += f"section: {result["section"]}\nquestion: {result["question"]}\nanswer: {result["text"]}\n\n"

prompt = prompt_template.format(question = question, context = context).strip()

prompt

"You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n\n    QUESTION: The course has already started, can I still enroll?\n\n    CONTEXT:\n    section: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection: General course-related questions\nquestion: Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.\n\

In [11]:

openai_client = OpenAI()

response = openai_client.chat.completions.create(
    model = "gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content


'Yes, you can still enroll in the course after it has already started. However, be aware that there will be deadlines for submitting the final projects, so it is advised not to leave everything until the last minute.'